# Notebook 08: Multi-Agent Debate & Mixture-of-Agents

## Learning Objectives
- Implement multi-agent debate (Du et al., 2023)
- Implement Mixture-of-Agents (MoA) (Wang et al., 2024)
- Compare: single agent vs debate vs MoA
- Visualize convergence over debate rounds

## Multi-Agent Debate

Key idea: agents argue and update based on others' responses.

```
Round 0: Agent1(Q), Agent2(Q), Agent3(Q)  ← independent
Round 1: Agent1(Q + A2_prev + A3_prev),   ← sees others
          Agent2(Q + A1_prev + A3_prev),
          Agent3(Q + A1_prev + A2_prev)
Converge when majority agree
```

## Mixture-of-Agents

```
Layer 1 (Proposers): Agent1(Q), Agent2(Q), Agent3(Q)  ← diverse, parallel
Layer 2 (Aggregator): Aggregator(Q + all_proposals)  ← synthesis
```

In [ ]:
# !pip install torch transformers

In [ ]:
import sys
sys.path.insert(0, '..')
from src.agents import SolverAgent
from src.orchestration import DebateOrchestrator, MixtureOfAgents
import json, matplotlib.pyplot as plt
print('Ready!')

## Step 1: Multi-Agent Debate

In [ ]:
agents = [SolverAgent(agent_id=f'solver_{i}') for i in range(3)]
debate = DebateOrchestrator(agents, num_rounds=3, convergence_threshold=0.67)
problem = 'A train travels 60 km/h for 3 hours. How far does it go?'
result = debate.run(problem, ground_truth=180.0)
print('Debate Result:')
print(f"  Final answer: {result['final_answer']}")
print(f"  Correct:      {result['correct']}")
print(f"  Rounds to converge: {result['rounds_to_converge']}")
print(f"  Agreement per round: {result['agreement_per_round']}")

## Step 2: Visualize Debate Convergence

In [ ]:
from src.evaluation.visualization import plot_debate_convergence
fig = plot_debate_convergence(
    result['agreement_per_round'],
    convergence_threshold=0.67,
    title='Debate Convergence: Train Distance Problem'
)
plt.show()

## Step 3: Mixture-of-Agents

In [ ]:
proposers = [SolverAgent(agent_id=f'proposer_{i}') for i in range(3)]
aggregator = SolverAgent(agent_id='aggregator')
moa = MixtureOfAgents(proposers, aggregator)
result_moa = moa.run(problem, ground_truth=180.0)
print('MoA Result:')
print(f"  Final answer: {result_moa['final_answer']}")
print(f"  Correct:      {result_moa['correct']}")
print('\nProposer responses:')
for i, r in enumerate(result_moa['proposer_responses']):
    print(f'  Proposer {i+1}: {r[:70]}')

---

## Exercises

1. **Debate rounds:** Try num_rounds=1, 2, 3, 5. When does accuracy plateau?
2. **Convergence threshold:** Try 0.5, 0.67, 1.0 (unanimous). Trade-off speed vs quality?
3. **MoA agents:** Try proposers=2, 3, 5. When does adding more agents stop helping?
4. **Disagreement analysis:** When agents disagree, which is more often right?
5. **Extension:** Implement a weighted vote (agents with higher past accuracy vote more)